In [1]:
!pip install datasets pandas pymongo sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 31.3 MB/s eta 0:00:00


In [2]:
!pip install -U transformers accelerate

### Loading the dataset using `datasets` library directly from hugging face

In [3]:
from datasets import load_dataset

dataset = load_dataset("MongoDB/embedded_movies", split="train")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

sample_mflix.embedded_movies.json:   0%|          | 0.00/42.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [4]:
import pandas as pd

In [5]:
data = pd.DataFrame(dataset)

In [6]:
data.head()

,plot,runtime,genres,fullplot,directors,writers,countries,poster,languages,cast,title,num_mflix_comments,rated,imdb,awards,type,metacritic,plot_embedding
0,Young Pauline is left a lot of money when her ...,199.0,[Action],Young Pauline is left a lot of money when her ...,"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...",[USA],https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",The Perils of Pauline,0,None,"{'id': 4465, 'rating': 7.6, 'votes': 744}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.0007293965299999999, -0.026834568000000003,..."
1,A penniless young man tries to save an heiress...,22.0,"[Comedy, Short, Action]",As a penniless man worries about how he will m...,"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],[USA],https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",From Hand to Mouth,0,TV-G,"{'id': 10146, 'rating': 7.0, 'votes': 639}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.022837115, -0.022941574000000003, 0.014937..."
2,"Michael ""Beau"" Geste leaves England in disgrac...",101.0,"[Action, Adventure, Drama]","Michael ""Beau"" Geste leaves England in disgrac...",[Herbert Brenon],"[Herbert Brenon (adaptation), John Russell (ad...",[USA],None,[English],"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",Beau Geste,0,None,"{'id': 16634, 'rating': 6.9, 'votes': 222}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.00023330492999999998, -0.028511643000000003..."
3,"Seeking revenge, an athletic young man joins t...",88.0,"[Adventure, Action]",A nobleman vows to avenge the death of his fat...,[Albert Parker],"[Douglas Fairbanks (story), Jack Cunningham (a...",[USA],https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",The Black Pirate,1,None,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[-0.005927917, -0.033394486, 0.0015323418, -0...."
4,An irresponsible young millionaire changes his...,58.0,"[Action, Comedy, Romance]","The Uptown Boy, J. Harold Manners (Lloyd) is a...",[Sam Taylor],"[Ted Wilde (story), John Grey (story), Clyde B...",[USA],https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",For Heaven's Sake,0,PASSED,"{'id': 16895, 'rating': 7.6, 'votes': 918}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.0059373598, -0.026604708, -0.0070914757000..."


In [7]:
data.columns

Index(['plot', 'runtime', 'genres', 'fullplot', 'directors', 'writers',
       'countries', 'poster', 'languages', 'cast', 'title',
       'num_mflix_comments', 'rated', 'imdb', 'awards', 'type', 'metacritic',
       'plot_embedding'],
      dtype='object')

In [8]:
data["plot"][0]

"Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle's secretary has been named as her guardian until she marries, at which time she will officially take ..."

In [9]:
data["fullplot"][0]

'Young Pauline is left a lot of money when her wealthy uncle dies. However, her uncle\'s secretary has been named as her guardian until she marries, at which time she will officially take possession of her inheritance. Meanwhile, her "guardian" and his confederates constantly come up with schemes to get rid of Pauline so that he can get his hands on the money himself.'

In [10]:
data["fullplot"].isnull().sum()

np.int64(48)

In [11]:
data.shape

(1500, 18)

In [12]:
data.isnull().sum()

,0
plot,27
runtime,15
genres,0
fullplot,48
directors,13
writers,13
countries,0
poster,89
languages,1
cast,1


In [13]:
dataset_df = data.dropna(subset=["fullplot"])

In [14]:
dataset_df["fullplot"].isnull().sum()

np.int64(0)

In [15]:
dataset_df = dataset_df.drop(columns=["plot_embedding"])

In [16]:
dataset_df.head(2)

,plot,runtime,genres,fullplot,directors,writers,countries,poster,languages,cast,title,num_mflix_comments,rated,imdb,awards,type,metacritic
0,Young Pauline is left a lot of money when her ...,199.0,[Action],Young Pauline is left a lot of money when her ...,"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...",[USA],https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",The Perils of Pauline,0,None,"{'id': 4465, 'rating': 7.6, 'votes': 744}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN
1,A penniless young man tries to save an heiress...,22.0,"[Comedy, Short, Action]",As a penniless man worries about how he will m...,"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],[USA],https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",From Hand to Mouth,0,TV-G,"{'id': 10146, 'rating': 7.0, 'votes': 639}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN


In [17]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("thenlper/gte-large")

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [18]:
def get_embedding(text: str):
  if not text.strip():
    print("Attempted to get embedding for empty string")
    return []
  embedding = embedding_model.encode(text)
  return embedding.tolist()

In [19]:
dataset_df["plot_embedding"] = dataset_df["fullplot"].apply(get_embedding)

### Connecting to MongoDB

In [21]:
from google.colab import userdata
mongo_db_uri = userdata.get('mongo_db_uri')

In [22]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri = mongo_db_uri

client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [23]:
db = client["movie_db"]

In [24]:
collection = db["collection01"]

In [25]:
document = dataset_df.to_dict("records")

In [27]:
collection.insert_many(document)

InsertManyResult([ObjectId('69846df9cd702624bedaa4b1'), ObjectId('69846df9cd702624bedaa4b2'), ObjectId('69846df9cd702624bedaa4b3'), ObjectId('69846df9cd702624bedaa4b4'), ObjectId('69846df9cd702624bedaa4b5'), ObjectId('69846df9cd702624bedaa4b6'), ObjectId('69846df9cd702624bedaa4b7'), ObjectId('69846df9cd702624bedaa4b8'), ObjectId('69846df9cd702624bedaa4b9'), ObjectId('69846df9cd702624bedaa4ba'), ObjectId('69846df9cd702624bedaa4bb'), ObjectId('69846df9cd702624bedaa4bc'), ObjectId('69846df9cd702624bedaa4bd'), ObjectId('69846df9cd702624bedaa4be'), ObjectId('69846df9cd702624bedaa4bf'), ObjectId('69846df9cd702624bedaa4c0'), ObjectId('69846df9cd702624bedaa4c1'), ObjectId('69846df9cd702624bedaa4c2'), ObjectId('69846df9cd702624bedaa4c3'), ObjectId('69846df9cd702624bedaa4c4'), ObjectId('69846df9cd702624bedaa4c5'), ObjectId('69846df9cd702624bedaa4c6'), ObjectId('69846df9cd702624bedaa4c7'), ObjectId('69846df9cd702624bedaa4c8'), ObjectId('69846df9cd702624bedaa4c9'), ObjectId('69846df9cd702624bedaa4

### Data Retreival

In [76]:
user_query = "Which one is a good horror movie based on story and title to watch and why?"

In [55]:
def vector_search(user_query, collection):
  query_embedding = get_embedding(user_query)
  if query_embedding is None:
    return "Invalid query or embedding is failed!"
  pipeline = [{
                "$vectorSearch": {
                "index": "vector_index",
                "queryVector": query_embedding,
                "path": "plot_embedding",
                "numCandidates": 150,
                "limit": 4,
                }

            },
              {
                "$project": {
                "fullplot": 1,
                "title": 1,
                "genres": 1,
                "score": {"$meta": "vectorSearchScore"},
                 }
            }]
  result = collection.aggregate(pipeline)
  return list(result)

In [57]:
vector_search(user_query, collection)

[{'_id': ObjectId('69846df9cd702624bedaa6ac'),
  'genres': ['Action', 'Horror', 'Thriller'],
  'fullplot': 'The "sematary" is up to its old zombie-raising tricks again. This time, the protagonists are Jeff Matthews, whose mother died in a Hollywood stage accident, and Drew Gilbert, a boy coping with an abusive stepfather.',
  'title': 'Pet Sematary II',
  'score': 0.9057483673095703},
 {'_id': ObjectId('69846df9cd702624bedaa947'),
  'genres': ['Action', 'Horror', 'Sci-Fi'],
  'fullplot': 'Ana, a young beautiful nurse finishes her day-shift at the hospital to return home to her beloved husband, they make love and sleep together. The next day, after her husband is killed by her neighbor next door, he suddenly comes back to life. She discovers the chaos happening in her neighborhood and escapes from her home. Soon after coming to her senses in the woods, she encounters a cop and other survivors, they decide to find safety in a mall. Soon more survivors come, and they learn that if they wa

In [58]:
def get_search_result(user_query, collection):
  knowledge = vector_search(user_query, collection)
  search_result = ""
  for result in knowledge:
        search_result += f"Title: { result.get('title', 'N/A') }, Plot: { result.get('fullplot', 'N/A') }\n"
  return search_result

In [69]:
retrieved_info = get_search_result(user_query, collection)
print(retrieved_info)

Title: Pet Sematary II, Plot: The "sematary" is up to its old zombie-raising tricks again. This time, the protagonists are Jeff Matthews, whose mother died in a Hollywood stage accident, and Drew Gilbert, a boy coping with an abusive stepfather.
Title: Dawn of the Dead, Plot: Ana, a young beautiful nurse finishes her day-shift at the hospital to return home to her beloved husband, they make love and sleep together. The next day, after her husband is killed by her neighbor next door, he suddenly comes back to life. She discovers the chaos happening in her neighborhood and escapes from her home. Soon after coming to her senses in the woods, she encounters a cop and other survivors, they decide to find safety in a mall. Soon more survivors come, and they learn that if they want to stay alive, they should stick together as the world is overrun by an army of undead. Can they survive the horror in this horrific global chaos? When there is no more room in hell, the dead will walk the earth
Ti

### Response Generation

In [60]:
!pip install -U huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 553.2/553.2 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface_hub 1.3.4
    Uninstalling huggingface_hub-1.3.4:
      Successfully uninstalled huggingface_hub-1.3.4


In [61]:
from google.colab import userdata
hugging_face_token = userdata.get('hugging_face_access_token')

In [64]:
from huggingface_hub import notebook_login

notebook_login()

In [74]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "google/gemma-2b-it"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/164 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [77]:
prompt = f"Query: {user_query}\nContinue to answer the query by using the Search Results:\n{retrieved_info}."

# Tokenize input
model_inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate output (use max_new_tokens to control length)
response = model.generate(**model_inputs, max_new_tokens=500, do_sample=True)

# Decode tokens back to text
output_text = tokenizer.batch_decode(response, skip_special_tokens=True)[0]

print(output_text)

Query: Which one is a good horror movie based on story and title to watch and why?
Continue to answer the query by using the Search Results:
Title: Pet Sematary II, Plot: The "sematary" is up to its old zombie-raising tricks again. This time, the protagonists are Jeff Matthews, whose mother died in a Hollywood stage accident, and Drew Gilbert, a boy coping with an abusive stepfather.
Title: Dawn of the Dead, Plot: Ana, a young beautiful nurse finishes her day-shift at the hospital to return home to her beloved husband, they make love and sleep together. The next day, after her husband is killed by her neighbor next door, he suddenly comes back to life. She discovers the chaos happening in her neighborhood and escapes from her home. Soon after coming to her senses in the woods, she encounters a cop and other survivors, they decide to find safety in a mall. Soon more survivors come, and they learn that if they want to stay alive, they should stick together as the world is overrun by an a